# **Methode 2**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

# Prétraitement du texte
text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Enregistrer le tokenizer
with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")


text = text[:100000]

Taille du vocabulaire : 6745


In [3]:
my_input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        my_input_sequences.append(my_n_gram_sequence)


In [4]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

y = np.array(tf.keras.utils.to_categorical(y, num_classes=vocab_size))

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, 60))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))
model.build(input_shape=(None, X.shape[1])) 
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 18164, 60)      │       404,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       126,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 6745)           │     1,018,495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,549,795 (5.91 MB)

 Trainable params: 1,549,795 (5.91 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 478s 837ms/step - accuracy: 0.0325 - loss: 6.9413
Epoch 2/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.0410 - loss: 6.0571
Epoch 3/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.0572 - loss: 5.8195
Epoch 4/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.0762 - loss: 5.5806
Epoch 5/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 839ms/step - accuracy: 0.1067 - loss: 5.2702
Epoch 6/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.1201 - loss: 5.0285
Epoch 7/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 839ms/step - accuracy: 0.1317 - loss: 4.8144
Epoch 8/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 839ms/step - accuracy: 0.1414 - loss: 4.6450
Epoch 9/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 839ms/step - accuracy: 0.1596 - loss: 4.4233
Epoch 10/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.1688 - loss: 4.2701
Epoch 11/50
568/568 ━━━━━━━━━━━━━━━━━━━━ 476s 838ms/step - accuracy: 0.1773 - loss: 4.1025
Epoch 12

In [8]:
input_text = "But i want to talk"
predict_next_words= 6

for _ in range(predict_next_words):
    token_list = tokenizer.texts_to_sequences([input_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    input_text += " " + output_word

print(input_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
But i want to talk a secret and at length they
